In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-27b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-27b-it",
    device_map="auto",
)

/home/lenovo/miniforge3/envs/gemma/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 12/12 [00:29<00:00,  2.50s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))

<bos>Write me a poem about Machine Learning.

In realms of data, vast and deep,
Where patterns hide and secrets sleep,
Machine Learning, a beacon bright,
Unveils the truth


In [1]:
from vllm import LLM, SamplingParams

# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=512)

# Create an LLM.
llm = LLM(model="google/gemma-2-27b", download_dir = ".model_cache/")
# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)


PermissionError: [Errno 13] Permission denied: '/home/lenovo/.cache/huggingface/hub/.locks/models--google--gemma-2-27b/9515f989c9c21d533f1ce21f89ba8a997217a950.lock'

In [7]:
from synth_prompts import ALL_PROMPTS
languag="DANISH"
prompts = ALL_PROMPTS
prompts.keys()

dict_keys(['ASSYMETRIC_SHORT_LONG_TASK_GENERATOR', 'ASSYMETRIC_SHORT_LONG_GENERATOR', 'ASSYMETRIC_SHORT_LONG_PLACEHOLDERS', 'ASSYMETRIC_LONG_SHORT_TASK_GENERATOR', 'ASSYMETRIC_LONG_SHORT_GENERATOR', 'ASSYMETRIC_LONG_SHORT_PLACEHOLDERS', 'ASSYMETRIC_SHORT_SHORT_TASK_GENERATOR', 'ASSYMETRIC_SHORT_SHORT_GENERATOR', 'ASSYMETRIC_LONG_LONG_TASK_GENERATOR', 'ASSYMETRIC_LONG_LONG_GENERATOR'])

In [12]:
outputs = llm.generate(prompts['ASSYMETRIC_SHORT_SHORT_TASK_GENERATOR'], sampling_params)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.44it/s, est. speed input: 188.03 toks/s, output: 23.14 toks/s]


In [13]:
outputs

[RequestOutput(request_id=7, prompt='Brainstorm a list of text matching tasks where both the queries and the groundtruth documents are very short (one or two\nsentences, even a short phrase).\nHere are a few examples:\n- Given a scientific paper title, retrieve the title of papers that cite the given paper.\n- Match a word with its definition.\n- Provided a notable person’s name, identify their occupation or achievement.\nYour output must always be a python list of strings only, with about 20 elements, and each element corresponds to a distinct\ntask in one sentence. Do not explain yourself or output anything else. Be creative!\n', prompt_token_ids=[2, 14181, 72258, 476, 1889, 576, 2793, 19947, 13333, 1570, 2145, 573, 33733, 578, 573, 4216, 48400, 10817, 708, 1508, 3309, 591, 785, 689, 1378, 108, 106016, 235269, 1693, 476, 3309, 20911, 846, 108, 4858, 708, 476, 2619, 8944, 235292, 108, 235290, 22546, 476, 12132, 4368, 4680, 235269, 37898, 573, 4680, 576, 14219, 674, 45403, 573, 2764, 4

In [8]:
import random
from synth_prompts import ALL_PROMPTS

# Hardcode language parameter for dynamic replacements
language = "DANISH"

# Function to replace placeholders with random choices
def fill_placeholders(prompt, placeholders):
    for placeholder, options in placeholders.items():
        replacement = random.choice(options)
        prompt = prompt.replace(f'{{{placeholder}}}', str(replacement))
    return prompt

# Sample and process prompts
def generate_sampled_prompts(prompts):
    sampled_prompts = {}

    for key, prompt in prompts.items():
        if 'PLACEHOLDERS' in key:
            continue  # Skip placeholder dictionaries; only process raw text
        
        # Check for corresponding placeholder dictionary
        placeholders_key = f"{key}_PLACEHOLDERS"
        placeholders = prompts.get(placeholders_key, {})

        # Add language to the placeholders
        placeholders['language'] = [language]

        # Fill the prompt with random placeholders
        sampled_prompts[key] = fill_placeholders(prompt, placeholders)

    return sampled_prompts

# Generate and print the prompts suitable for the model
sampled_prompts = generate_sampled_prompts(ALL_PROMPTS)
for key, prompt in sampled_prompts.items():
    print(f"Sampled prompt for {key}:\n{prompt}\n")

Sampled prompt for ASSYMETRIC_SHORT_LONG_TASK_GENERATOR:
Brainstorm a list of potentially useful text retrieval tasks.
Here are a few examples for your reference:
- Retrieve relevant documents for a short keyword web search query that asks for weather information.
- Search for documents that answers a FAQ-style query on children’s nutrition.
Please adhere to the following guidelines:
- Specify what the query is, and what the desired documents are.
- Each retrieval task should cover a wide range of queries, and should not be too specific.
Your output must always be a python list of strings only, with about 20 elements, and each element corresponds to a distinct
retrieval task in one sentence. Do not explain yourself or output anything else. Be creative!


Sampled prompt for ASSYMETRIC_SHORT_LONG_GENERATOR:
You have been assigned a retrieval task: {task}
Your mission is to write one text retrieval example for this task in JSON format. The JSON object must contain the following
keys:
- "u